<a href="https://colab.research.google.com/github/CoolAdito/Colab-Sheet1/blob/master/AT_Lesson_23_Class_Copy_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 23: Meteorite Landings - Treating The Missing Values

### Teacher-Student Activities

In the previous class, we learnt how to slice a DataFrame by some logical conditions and also by using the Ampersand logical operator to combine two conditions as part of the data cleaning step. In this class, we will continue with the data cleaning process so that later we can make cartograms and distribution plots.

While treating the missing values we need to keep in mind two things:

1. Whether it is really required to replace the missing values with some new most appropriate value. In some specific problem statements, the missing values itself act as a critical value. 

2. The replaced value should not disrupt the original data otherwise, the actual results will get altered. Our logic of replacing the missing values with a new value must be consistent with the problem statement.

Let's quickly rush through the activities we covered in the previous class and begin this class from **Activity 1: Check For The Missing Values** section.



---

#### The Data

The dataset contains the following variables:

1. `name`: the name of the place where a meteorite was found or observed.

2. `id`: a unique identifier for a meteorite.

3. `nametype`: one of the following:
    
    - `valid`: a typical meteorite.
    
    - `relict`: a meteorite that has been highly degraded by the weather on Earth.

4. `recclass`: the class of the meteorite; one of a large number of classes based on physical, chemical, and other characteristics. 

5. `mass:` the mass of the meteorite, in grams

6. `fall`: whether the meteorite was seen falling, or was discovered after its impact; one of the following:

    - `Fell`: the meteorite's fall was observed.
    
    - `Found`: the meteorite's fall was not observed.

7. `year`: the year the meteorite fell, or the year it was found (depending on the value of fell).

8. `reclat`: the latitude of the meteorite's landing.

9. `reclong`: the longitude of the meteorite's landing.

10. `GeoLocation`: a parentheses-enclose, comma-separated tuple that combines `reclat` and `reclong` values.


---

#### Loading The Dataset

Dataset Link: https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/meteorite-landings/meteorite-landings.csv


In [ ]:
# Import the necessary libraries for this class and create a DataFrame.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

met_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/meteorite-landings/meteorite-landings.csv')
met_df.head()

In [ ]:
# Find the number of rows and columns in the DataFrame. 
met_df.shape

---

#### The `describe()` Function


In [ ]:
# Descriptive statistics summary of the 'year' values in the 'met_df' DataFrame. 
met_df['year'].describe()

---

#### Slicing A DataFrame And The Ampersand (`&`) Logical Operator

**Syntax:** `data_frame[condition1 & condition2 & condition3 ... conditionN]`

where `N` is the total number of conditions to be applied.


In [ ]:
# Rows containing the year values less than 860 and greater than 2016.
correct_years_df = met_df[(met_df['year'] >= 860) & (met_df['year'] <= 2016)]
correct_years_df

---

#### Removing The Invalid `reclong` Values


In [ ]:
# Rows having the 'reclong' values greater than or equal to -180 degrees and less than or equal to 180 degrees.
correct_long_df = correct_years_df[(correct_years_df['reclong'] >= -180) & (correct_years_df['reclong'] <= 180)]
correct_long_df

---

#### Removing The Rows Containing `0 N, 0 E` Values


In [ ]:
# Rows containing the 0 'reclat' and 0 'reclong' values from the 'correct_long_df'.
correct_lat_long_df = correct_long_df[~((correct_long_df['reclat'] == 0 ) & (correct_long_df['reclong'] == 0))]
correct_lat_long_df

---

#### Activity 1: Check The Missing Values^^

So far we have removed all the unwanted values or rows. Now it's time to check whether we have any missing values or not. The missing values are generally reported as `NaN` values. We can find the rows or columns containing the `NaN` values using either the `isnull()` or the `isna()` function. They both return `True` for the `NaN` (or null or missing) values.

Let's first check whether any column has a missing value or not. 

In [ ]:
# Student Action: Check whether the 'correct_lat_long_df' DataFrame has missing values or not using the 'isnull()' function.


So, there are 107 null values in the `mass` column. Let's also check for the same using the `isna()` function.

In [ ]:
# Student Action: Check whether the 'correct_lat_long_df' DataFrame has missing values or not using the 'isna()' function.


Again, we can see that there are 107  missing values in the `correct_lat_long_df` DataFrame. Let's retrieve all the rows containing the missing values. For this, 

- we will write the name of the variable storing the DataFrame followed by square brackets. 

    ```
    correct_lat_long_df[]
    ```

- Inside the square brackets, we will use the `isnull()` function to get all the rows in which the `mass` value is `NaN`.

    ```
    correct_lat_long_df[correct_lat_long_df['mass'].isnull() == True]
    ```

In [ ]:
# Student Action: Retrieve all the rows containing the missing 'mass' values in the 'correct_lat_long_df' DataFrame.


Because of the condition `correct_lat_long_df['mass'].isnull() == True`, the `correct_lat_long_df` DataFrame returns all the rows containing the `NaN` mass values.

We are not going to remove these rows. Instead, we will replace all the `NaN` values in the `mass` column with the median mass value because most of the `mass` values lie between the first and the third quartile values. The second quartile (or median) is a fair representative of the values lying in the inter-quartile range, i.e., $(Q3 - Q1)$.

More importantly, the quartile values remain unaffected by the unusually very high or very low values. In other words, the outliers do not affect the quartile values because they are computed by arranging all the values in the increasing order.

Now, let's get the descriptive statistics for the `mass` column in the `correct_lat_long_df` DataFrame.

In [ ]:
# Student Action: Get descriptive statistics for the 'mass' column in the 'correct_lat_long_df' DataFrame.


The median `mass` value is `29.6` grams. We will replace all `NaN` values in the `mass` column with `29.6`. After replacement, all the above rows in the `mass` column should have `29.6` value.

---

#### Activity 2: The `index` Keyword

Let's create an array containing the indices of the rows having the `NaN` values in the `mass` column. To get an array containing the row indices of a whole DataFrame, use the `index` keyword of the Pandas module.

In [ ]:
# Teacher Action: Create a list of the indices of above rows.


So, here we have an array containing the indices of all the rows having the `NaN` mass values. There are 107 such rows.

---

#### Activity 3: The `loc[]` Function^

Now, let's replace the missing `mass` values with the median of `mass` values. For this, we need to use the `loc[]` function from Pandas library. It is an extension to the `iloc[]` function. The `iloc[]` function takes only the row and column indices as inputs. But in addition to the row and column indices, the `loc[]` function also accepts the array of row indices and column indices as inputs. It also accepts the specific column names as inputs.

Let's create a Pandas series containing all the rows having the `NaN` mass values using the `loc[]` function. We will pass `row_indices` array and `mass` column as inputs to the `loc[]` function. Let's store the series obtained in the `missing_mass_values` variable.

In [ ]:
# Student Action: Retrieve the missing 'mass' values from 'correct_lat_long_df' DataFrame using the 'loc[]' function. 


Here, we have a Pandas series which contains all the `NaN` mass values. The length of this series is `107`.

---

#### Activity 4: Replacing The Missing `mass` Values^^^

Now, let's replace the missing `mass` values in the 107 rows with the median of the `mass` values. Again, we will use the `loc[]` function for this exercise.

Using the `loc[]` function with `row_indices` and `mass` as inputs, we will get all the `NaN` values in the `mass` column. Then, using the `median()` function, we will get the median value which we are going to assign to the rows having the missing `mass` values in the `correct_lat_long_df`.

**Note:** Python will throw some warning after running the code below. Ignore it!

In [ ]:
# Teacher Action: Replace the missing values in the 'mass' column in the 'correct_lat_long_df' DataFrame with median of mass.


Now, let's check whether all the missing mass values have been replaced by the median of the mass values or not. We can do this with the help of the `loc[]` function. 

In [ ]:
# Student Action: Check whether all the missing mass values have been replaced by the median of the mass values or not.


Each item in the `mass` column in the above DataFrame of 107 rows is `29.6` grams.

Let's confirm it by getting the descriptive statistics for the `mass` column in the above DataFrame using the `describe()` function. Except for the `std` and `count` values, all other values should be `29.6`. The `std` value should be very very close to `0` because every value will be the same as the mean value. The `count` value should be `107`.

In [ ]:
# Student Action: Create a descriptive statistics summary of the 'mass' column in the above DataFrame containing 107 rows.


As expected, the `count` value is `107`, the `std` value is very close to `0` because $10^{-14}$ or $\frac{1}{10^{14}}$ is practically zero. 

Now, let's get the descriptive statistics for the entire `mass` column in `correct_lat_long_df`.

In [ ]:
# Student Action: Create a descriptive statistics summary of the 'mass' column in the 'correct_lat_long_df' DataFrame.


As you can see there is no major change in the `std, min, 25%, 50%, 75%` and `max` values. They all are the same which is how it should be. **Treatment of the missing values should not affect the original dataset drastically.**

In the next class, we will take a pause from the data cleaning process and learn the basics of cartograms. In the subsequent class, we will create cartograms for the meteorite landing sites on Earth.

---